In [167]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import json
import time
from pymongo import MongoClient

In [168]:
client = MongoClient("localhost:27017")
db = client["geoSpatial-data-project"]

In [169]:
load_dotenv()
token = os.getenv("token")

In [170]:
def extract_from_one_element(venue):
    name = venue["name"]
    distance = venue["distance"]
    zip_code = venue["location"]["postcode"]
    location =  [float(venue["geocodes"]["main"]["latitude"]), float(venue["geocodes"]["main"]["longitude"])]
    
    small_dict = {
        "name": name,
        "distance": distance,
        "zip_code": zip_code,
        "location": location
    }
    return small_dict

def foursquare_request(query, lat, lon, radius):
    
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit=50"
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }  
    response = requests.get(url, headers=headers)
    response_json = response.json()

    if "%20" in query:
        query = query.replace("%20", "_")
    
    venues = [extract_from_one_element(i) for i in response_json["results"]]
    df = pd.DataFrame(venues)
    df["Type"] = query
    return df

def concat_requests(city, venues):
    dfs = []
    if city == "San Francisco":
        lat = 37.773972
        lon = -122.431297
    elif city == "New York":
        lat = 40.71427
        lon = -74.00597
    elif city == "Palo Alto":
        lat = 37.43504
        lon = -122.13351
    for i in venues:
        df = foursquare_request(i, lat, lon, 6000)
        dfs.append(df)
    df_venues = pd.concat(dfs, ignore_index=True)
    return df_venues


In [171]:
venues = ["starbucks", "school", "night%20club", "basquetball", "pet", "basketball%20court", "pet%20grooming", "vegan"]

In [173]:
sf_df = concat_requests("San Francisco", venues)
sf_df.to_csv("../data/sf_venues.csv", index=False)
db['sf_venues'].insert_many(sf_df.to_dict("records"))

sf_df

,name,distance,zip_code,location,Type
0,Starbucks,1103.0,94102,"[37.777222, -122.41935]",starbucks
1,Starbucks,1598.0,94109,"[37.785916, -122.421011]",starbucks
2,Starbucks,1476.0,94114,"[37.761006, -122.434724]",starbucks
3,Starbucks,1659.0,94115,"[37.788569, -122.434572]",starbucks
4,Starbucks,974.0,94115,"[37.782793, -122.431506]",starbucks
...,...,...,...,...,...
333,Mission Bowling Club,1716.0,94110,"[37.763834, -122.416696]",vegan
334,SOMA StrEat Food Park,1782.0,94103,"[37.769745, -122.412028]",vegan
335,Beretta Valencia,2423.0,94110,"[37.75383, -122.42062]",vegan
336,Ritual Coffee Roasters,2140.0,94110,"[37.756454, -122.421348]",vegan


In [172]:
ny_df = concat_requests("New York", venues)
ny_df.to_csv("../data/ny_venues.csv", index=False)
db['ny_venues'].insert_many(ny_df.to_dict("records"))
ny_df

,name,distance,zip_code,location,Type
0,Starbucks,797.0,10281,"[40.714207, -74.01571]",starbucks
1,Starbucks,361.0,10038,"[40.711054, -74.004761]",starbucks
2,Starbucks,454.0,10013,"[40.718154, -74.007443]",starbucks
3,Starbucks,286.0,10013,"[40.715842, -74.003205]",starbucks
4,Starbucks,65.0,10007,"[40.714957, -74.006007]",starbucks
...,...,...,...,...,...
327,787 Coffee,1266.0,10004,"[40.703449, -74.010743]",vegan
328,Ground Support,1159.0,10012,"[40.724345, -74.001926]",vegan
329,Mudspot,2308.0,10003,"[40.729074, -73.986657]",vegan
330,Breads Bakery,2754.0,10003,"[40.736592, -73.99193]",vegan


In [174]:
pa_df = concat_requests("Palo Alto", venues)
pa_df.to_csv("../data/pa_venues.csv", index=False)
db['pa_venues'].insert_many(pa_df.to_dict("records"))
pa_df

,name,distance,zip_code,location,Type
0,Starbucks,1566.0,94303,"[37.447938, -122.126298]",starbucks
1,Starbucks,1628.0,94306,"[37.427314, -122.149185]",starbucks
2,Starbucks,1303.0,94306,"[37.426804, -122.144077]",starbucks
3,Starbucks,1984.0,94306,"[37.417438, -122.13053]",starbucks
4,Starbucks,2963.0,94303,"[37.461688, -122.137232]",starbucks
...,...,...,...,...,...
199,Sweetgreen,2925.0,94304,"[37.411713, -122.148881]",vegan
200,Sweetgreen,3121.0,94304,"[37.408569, -122.145355]",vegan
201,Sweetgreen,3775.0,94304,"[37.435732, -122.176287]",vegan
202,Sweetgreen,3877.0,94022,"[37.402753, -122.11685]",vegan
